In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio
import altair as alt


# mapping to the modules that make the app
sys.path.insert(0, "../..")
# sys.path.insert(0, "..")


In [6]:
from app.creators import universe
from app.functions import maths

from notebooks.helpers import test_queries

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import app.models as models

c = models.CosmosdbClient()

executing local windows deployment


In [4]:
buildings = yaml.safe_load(
    open(os.path.join(os.getenv("abspath"), "app", "configurations", "buildings.yaml"))
)
buildings["explanation"]

[{'type': "all buildings have a 'type'/'label' of building"},
 {'name': "name_with_underscore. UI can remove the '_' at display time."},
 {'augments_resource': 'OPTIONAL augments a specific resource, creates it if it does not exist'},
 {'built_by': 'type of object that can build the building'},
 {'owned_by': 'type of object that can own/maintain this building, assuming a relationhsip to that object'},
 {'changes': 'stats changed by the owner'},
 {'faction_augments': 'if owned by a pop in the faction,  raises the stat for all pops in factions'},
 {'effort': 'time in `universal time units` that the action will take to complete'},
 {'requires_attr': 'required attributes of the agent. Determines if the agent can do the action.'},
 {'render_type': "guides the rendering in buildings.js. Available types: 'block'"}]

## Adding a building to a pop, for testing. 

On a newly created population, adding a building to it so that I can test it. 
[refer to the ontology](../../docs/img/Exodestiny_Ontology.png)

In [5]:
farm = buildings["buildings"][0]
farm["objid"] = maths.uuid()
farm["label"] = "building"
farm

{'name': 'farmland',
 'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
 'planet_requirements': {'isHabitable': 'true'},
 'augments_resource': {'vegitable_foodstuff': 10},
 'changes': {'wealth': -0.01},
 'owned_by': 'pop',
 'effort': 1,
 'render_type': 'block',
 'objid': '2929492505497',
 'label': 'building'}

In [15]:
username = "BillmanLocal2"
pop = test_queries.get_random_pop(c,username)
pop


{'name': 'Obattla Arteskreis',
 'objid': '4877750744555',
 'conformity': 0.509,
 'literacy': 0.35,
 'aggression': 0.66,
 'constitution': 0.488,
 'health': 0.42,
 'isIn': '8822366435138',
 'industry': 0.343,
 'wealth': 0.196,
 'factionLoyalty': 0.414,
 'isIdle': 'true',
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '4877750744555'}

I'm arbitrarily adding a farm to the _ownership_ of this building. 

In [16]:
edge = {"node1": farm["objid"], "node2": pop['objid'], "label": "owns"}
edge

{'node1': '2929492505497', 'node2': '4877750744555', 'label': 'owns'}

In [17]:
data = {"nodes": [farm], "edges": [edge]}
data

{'nodes': [{'name': 'farmland',
   'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
   'planet_requirements': {'isHabitable': 'true'},
   'augments_resource': {'vegitable_foodstuff': 10},
   'changes': {'wealth': -0.01},
   'owned_by': 'pop',
   'effort': 1,
   'render_type': 'block',
   'objid': '2929492505497',
   'label': 'building'}],
 'edges': [{'node1': '2929492505497',
   'node2': '4877750744555',
   'label': 'owns'}]}

In [18]:
c.upload_data(username, data)

In [21]:
building_query = f"""
g.V().has('objid','{pop['objid']}').as('pop')
    .in('owns').as('building')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('objid','name','changes','augments_resource','planet_requirements','description','render_type'))
"""

c.run_query(building_query)
c.res

[{'labels': [['pop'], ['building']],
  'objects': [{'objid': ['4877750744555'], 'name': ['Obattla Arteskreis']},
   {'objid': ['2929492505497'],
    'name': ['farmland'],
    'changes': ['{wealth: -0.01}'],
    'augments_resource': ['{vegitable_foodstuff: 10}'],
    'planet_requirements': ['{isHabitable: true}'],
    'description': ['Generates organic foodstuffs, populations will consume food before consuming natural resources'],
    'render_type': ['block']}]}]